# Package Imports


In [ ]:
import requests
import json
import urllib3
from typing import List
from configparser import ConfigParser
import boto3
from boto3.dynamodb.conditions import Key

# Get the IBP URL and credentials and the run the functions below. Please load the functions before running



In [ ]:
    secret_name = "IBPCred"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    # Decrypts secret using the associated KMS key.
    
    #secret = get_secret_value_response['SecretString']
    sapauth = json.loads(get_secret_value_response['SecretString'])
    
    dynamodb = boto3.resource('dynamodb')

    table = dynamodb.Table('ibprequests')

    response = table.query(IndexName="Status-index",
                           KeyConditionExpression=Key('Status').eq('Initial'),
                            )
    print(type(response))
    requestlist = response['Items']
    for i in requestlist:
        print(i['RequestID'])
        process_forecast_request(int(i['RequestID']),sapauth)
   

        
    
    
    
    
    

# Process forecast function for each IBP Request- from SAP OSS note 3170544  

In [ ]:
def process_forecast_request(request_id: int, auth: dict):
    """Runs the forecasting process on the request.

    Request JSON:
    {
        "RequestID": Integer,
        "AlgorithmName": String,
        "AlgorithmParameter": String("ParameterName, ParameterType("integer", "double", "text"), ParameterValue"),
        "HistoricalPeriods": Integer,
        "ForecastPeriods": Integer,
    }

    Data JSON:
    {
        "@odata.context": String containing the metadata information,
        "@odata.metadataEtag": String containing the aggregation level, and datetime stamp. "W/\"20220303095028\"",
        "value":
        [
            {
                "RequestID": Integer,
                "GroupID": Integer,
                "_AlgorithmDataInput": 
                    [
                        {
                            "RequestID": Integer, 
                            "GroupID": Integer, 
                            "SemanticKeyFigure": "HISTORY" or "INDEPENDENT001" - "INDEPENDENT999", 
                            "TimeSeries": Timeseries in string format with ";" separator character,
                        }
                    ], 
                "_MasterData": 
                    [
                        {
                            "RequestID": Integer, 
                            "GroupID": Integer, 
                            "AttributeID": Planning attribute name string, 
                            "AttributeValue": Planning attribute value string,
                        }
                    ]
            }
        ]
        "@odata.nextLink": String containing the next url path to the next data chunk,
    }

    Results JSON:
    {
        "RequestID": Integer,
        "_AlgorithmDataOutput": [
            {
                "RequestID": Integer,
                "GroupID": Integer,
                "SemanticKeyFigure": "EXPOST" or "FORECAST",
                "ResultData": List of result strings,
            }
        ],
        "_Message": [
            {
                "RequestID": Integer,
                "GroupID": Integer,
                "MessageSequence": Integer,
                "MessageType": "I" or "E",
                "MessageText": String,
            }
        ]
    }

    Args:
        request_id (int): ID number of the request to be processed

    """
    
    # User credentials and server url
    SERVER_URL = auth["URL"]
    USERNAME = auth["Username"]
    PASSWORD = auth["Password"]
    
    
    
    
    TASK_URL = f"https://{SERVER_URL}/sap/opu/odata4/ibp/api_dmdfcstextalg/srvd_a2x/ibp/api_dmdfcstextalg/0001/Request"
    RESULT_URL = f"https://{SERVER_URL}/sap/opu/odata4/ibp/api_dmdfcstextalg/srvd_a2x/ibp/api_dmdfcstextalg/0001/Result"
    DATA_URL = f"https://{SERVER_URL}/sap/opu/odata4/ibp/api_dmdfcstextalg/srvd_a2x/ibp/api_dmdfcstextalg/0001/Input"
    
    print(
        f"Processing forecast request for id {request_id} started.", flush=True)

    request_get = requests.get(f"{TASK_URL}?$filter=RequestID%20eq%20{request_id}",
                               headers={"accept": "application/json"}, auth=(USERNAME, PASSWORD), verify=False)
    
    if request_get.status_code == 200:
        request_data = request_get.json()["value"][0]
        cookies = request_get.cookies

        algorithm_name = request_data["AlgorithmName"]

        parameters = {}
        if request_data["AlgorithmParameter"]:
            param_list = []
            for parameter in request_data["AlgorithmParameter"].split(";"):
                if "=" in parameter:
                    param_list.append(map(str.strip, parameter.split('=', 1)))
            parameters = dict(param_list)

        historical_periods = request_data["HistoricalPeriods"]
        forecast_periods = request_data["ForecastPeriods"]

        data_get = requests.get(f"{DATA_URL}?$filter=RequestID%20eq%20{request_id}&$expand=_AlgorithmDataInput,_MasterData",
                                headers={"accept": "application/json"}, cookies=cookies, verify=False)

        results = {}
        if data_get.status_code == 200:
            planning_objects = data_get.json()["value"]
            if "@odata.nextLink" in data_get.json().keys():
                try:
                    planning_objects = get_remaining_data(
                        planning_objects, f"https://{SERVER_URL}{data_get.json()['@odata.nextLink']}", cookies)
                except ConnectionError:
                    return

            # Output format
            output = {
                "RequestID": request_id,
                "_AlgorithmDataOutput": [],
                "_Message": [],
            }

            for planning_object in planning_objects:
                results = calculate_forecast(
                    planning_object, algorithm_name, parameters, historical_periods, forecast_periods)

                if len(results.keys()):
                    # Expost
                    expost_data = {
                        "RequestID": request_id,
                        "GroupID": planning_object["GroupID"],
                        "SemanticKeyFigure": "EXPOST",
                        "ResultData": results["EXPOST"],
                    }
                    output["_AlgorithmDataOutput"].append(expost_data)

                    # Forecast
                    forecast_data = {
                        "RequestID": request_id,
                        "GroupID": planning_object["GroupID"],
                        "SemanticKeyFigure": "FORECAST",
                        "ResultData": results["FORECAST"],
                    }
                    output["_AlgorithmDataOutput"].append(forecast_data)

                    # Message
                    message = {
                        "RequestID": request_id,
                        "GroupID": planning_object["GroupID"],
                        "MessageSequence": 1,
                        "MessageType": "I",
                        "MessageText": "Okay",
                    }
                    output["_Message"].append(message)

            # Header message
            if len(results.keys()):
                msg_header = {
                    "RequestID": request_id,
                    "GroupID": -1,
                    "MessageSequence": 1,
                    "MessageType": "I",
                    "MessageText": f"{algorithm_name}",
                }
                output["_Message"].append(msg_header)
            else:
                msg_header = {
                    "RequestID": request_id,
                    "GroupID": -1,
                    "MessageSequence": 1,
                    "MessageType": "E",
                    "MessageText": f"Forecast calculation failed! Algorithm: {algorithm_name} .",
                }
                output["_Message"].append(msg_header)

            output_json = json.dumps(output)

            token_request = requests.get(RESULT_URL, headers={"x-csrf-token": "fetch", "accept": "application/json"},
                                         cookies=cookies, verify=False)

            if token_request.status_code == 200:

                result_send_post = requests.post(RESULT_URL, str(output_json), cookies=cookies,
                                                headers={"x-csrf-token": token_request.headers["x-csrf-token"],
                                                        "Content-Type": "application/json", "OData-Version": "4.0"}, verify=False)

                print(
                    f"Forecast result for id {request_id} sent back to IBP system! Status code: {result_send_post.status_code}.", flush=True)
            else:
                print(
                f"Failed to retrieve x-csrf token! Status code: {token_request.status_code}.", flush=True)
        else:
            print(
                f"Failed to retrieve forecast data! Status code: {data_get.status_code}.", flush=True)
    else:
        print(
            f"Failed to retrieve forecast model details! Status code: {request_get.status_code}", flush=True)

# Execute forecast

In [ ]:
from typing import List, Dict

def create_result_string(results: List) -> str:
    """Concatenates the results with ';' to a string.

    Args:
        results (List): List of the algorithm results

    Returns:
        str: Concatenated results
    """

    return f"{''.join(str(x)+';' for x in results)}"[:-1]


def average_calulation(planning_object_data: Dict, historical_periods: int, forecast_periods: int) -> Dict:
    """Calculates Average Forecast

    Args:
        planning_object_data (Dict): Dictionary containing the timeseries of the planning object
        historical_periods (int): Historical periods number
        forecast_periods (int): Forecast periods number

    Returns:
        Dict: Dictionary containing the expost and forecast
    """

    mean_value = sum(planning_object_data["HISTORY"]) / historical_periods

    expost = historical_periods * [mean_value]
    forecast = forecast_periods * [mean_value]

    return {"EXPOST": create_result_string(expost), "FORECAST": create_result_string(forecast)}


def weighted_moving_average_calculation(planning_object_data: Dict, parameters: Dict, historical_periods: int, forecast_periods: int) -> Dict:
    """Calculates Average Forecast

    Args:
        planning_object_data (Dict): Dictionary containing the timeseries of the planning object
        parameters (Dict): Dictionary of the algorithm parameters
        historical_periods (int): Historical periods number
        forecast_periods (int): Forecast periods number

    Returns:
        Dict: Dictionary containing the expost and forecast
    """

    demand = planning_object_data["HISTORY"]
    window = int(parameters["Window"])
    weights = []
    if len(planning_object_data["INDEPENDENT001"]) == historical_periods:
        weights = planning_object_data["INDEPENDENT001"] + \
            [planning_object_data["INDEPENDENT001"][-1]] * forecast_periods
    else:
        weights = planning_object_data["INDEPENDENT001"]

    weighted_past = [x*w for x, w in zip(demand, weights[:historical_periods])]

    sumed_past_moving_windows = []
    sumed_weight_moving_windows = []

    for index in range(0, historical_periods - window + 1):
        sumed_past_moving_windows.append(
            sum(weighted_past[index: index + window]))
        sumed_weight_moving_windows.append(
            sum(weights[index: index + window]))

    result = [x/w for x,
              w in zip(sumed_past_moving_windows, sumed_weight_moving_windows)]

    expost = ["NULL"]*window + list(result[:-1])

    forecast = []
    if "Extend" in parameters.keys():

        demand.append(result[-1])
        for i in range(historical_periods, historical_periods+forecast_periods-1):
            weighted_past.append(demand[-1] * weights[i])
            result.append(sum(
                weighted_past[-window:]) / sum(weights[i-window+1:i+1]))
            demand.append(result[-1])

        forecast = result[-forecast_periods:]

    else:
        forecast = [result[-1]] * forecast_periods

    return {"EXPOST": create_result_string(expost), "FORECAST": create_result_string(forecast)}


def calculate_forecast(planning_object: Dict, alogrithm_name: str, parameters: Dict,
                       historical_periods: int, forecast_periods: int) -> Dict:
    """Forecast calculation function

    Args:
        planning_object (Dict): Dictionary of the timeseries for one planning object
        alogrithm_name (str): Name of the algortihm
        parameters (Dict): Dictionary of the algorithm parameters
        historical_periods (int): Number of historical periods
        forecast_periods (int): Number of forecast periods

    Returns:
        Dict: Dictionry of result strings
    """

    planning_object_data = {}

    for data in planning_object["_AlgorithmDataInput"]:
        if data["SemanticKeyFigure"] == "HISTORY":
            planning_object_data.update(
                {"HISTORY": [float(x) for x in data["TimeSeries"].split(';')][:historical_periods]})

        else:
            planning_object_data.update(
                {data["SemanticKeyFigure"]: [float(x) for x in data["TimeSeries"].split(';')]})

    results = {}

    if alogrithm_name == "Sagemaker":

        results = average_calulation(
            planning_object_data, historical_periods, forecast_periods)

    elif alogrithm_name == "Weighted MA":

        results = weighted_moving_average_calculation(
            planning_object_data, parameters, historical_periods, forecast_periods)

    return results
